[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb)

In [42]:
import os
!export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ.pop('TF_ENABLE_WINOGRAD_NONFUSED', None)

'1'

In [1]:
from google.colab import files
uploaded = files.upload()

Saving fer2013.tar to fer2013.tar


In [0]:
!tar -xf fer2013.tar

In [0]:

import csv
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization 
from keras.utils import np_utils

In [0]:
img_rows, img_cols = 48, 48
batch_size = 128
nb_classes = 7
nb_epoch = 1200
img_channels = 1

In [0]:

f = open('fer2013/fer2013.csv')
csv_f = csv.reader(f)

train_x = []
train_y = []
val_x =[]
val_y =[]


In [0]:
num=0
for row in csv_f:
  num = num +1
  #print(num)
  temp_list = []
  for pixel in row[1].split( ):
    temp_list.append(int(pixel))

  if str(row[2]) == "Training":
    train_y.append(int(row[0]))
    train_x.append(temp_list) 
  elif str(row[2]) == "PublicTest":
    val_y.append(int(row[0]))
    val_x.append(temp_list) 

In [0]:

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

In [0]:
train_x = train_x.reshape(train_x.shape[0], 48, 48)
train_x = train_x.reshape(train_x.shape[0], 1, 48, 48)
train_y = np_utils.to_categorical(train_y, 7)

In [0]:
val_x = val_x.reshape(val_x.shape[0], 48, 48)
val_x = val_x.reshape(val_x.shape[0], 1, 48, 48)
val_y = np_utils.to_categorical(val_y, 7)

In [44]:
input_shape = (img_channels, img_rows, img_cols)
model = Sequential()
model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same',
                            name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
#model.add(Dropout(.5))

model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.2))

model.add(Convolution2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(Convolution2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(Flatten())

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.45))

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.45))

model.add(Dense(output_dim=7))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7)`


In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (SeparableConv2D (None, 1, 48, 64)         3568      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 48, 64)         256       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 48, 64)         0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 1, 48, 64)         4736      
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 48, 64)         256       
_________________________________________________________________
activation_2 (Activation)    (None, 1, 48, 64)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 24, 64)         0         
__________

In [0]:
model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [18]:
# Wino 1 Separable
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 10s 351us/step - loss: 1.7963 - acc: 0.3183 - val_loss: 2.9346 - val_acc: 0.2817
Epoch 2/10
28709/28709 [==============================] - 10s 347us/step - loss: 1.6831 - acc: 0.3589 - val_loss: 1.6419 - val_acc: 0.3717
Epoch 3/10
28709/28709 [==============================] - 10s 342us/step - loss: 1.5985 - acc: 0.3851 - val_loss: 1.5624 - val_acc: 0.3901
Epoch 4/10
  640/28709 [..............................] - ETA: 9s - loss: 1.5430 - acc: 0.4125

28709/28709 [==============================] - 10s 342us/step - loss: 1.5402 - acc: 0.4061 - val_loss: 1.5583 - val_acc: 0.4168
Epoch 5/10
28709/28709 [==============================] - 10s 343us/step - loss: 1.4875 - acc: 0.4281 - val_loss: 1.4518 - val_acc: 0.4347
Epoch 6/10
28709/28709 [==============================] - 10s 344us/step - loss: 1.4373 - acc: 0.4457 - val_loss: 1.5184 - val_acc: 0.4271
Epoch 7/10
12160/28709 [===========>..................] - ETA: 5s - loss: 1.3944 - acc: 0.4608

28709/28709 [==============================] - 10s 344us/step - loss: 1.4016 - acc: 0.4594 - val_loss: 1.4820 - val_acc: 0.4302
Epoch 8/10
28709/28709 [==============================] - 10s 342us/step - loss: 1.3703 - acc: 0.4740 - val_loss: 1.3949 - val_acc: 0.4661
Epoch 9/10
28709/28709 [==============================] - 10s 339us/step - loss: 1.3437 - acc: 0.4833 - val_loss: 1.4676 - val_acc: 0.4570
Epoch 10/10
14208/28709 [=============>................] - ETA: 4s - loss: 1.3064 - acc: 0.4942

28709/28709 [==============================] - 10s 342us/step - loss: 1.3101 - acc: 0.4956 - val_loss: 1.3927 - val_acc: 0.4553
--- 98.65123987197876 seconds ---


In [23]:
# Wino 0 Separable
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 13s 446us/step - loss: 1.9749 - acc: 0.2584 - val_loss: 1.7371 - val_acc: 0.3118
Epoch 2/10
28709/28709 [==============================] - 10s 344us/step - loss: 1.7895 - acc: 0.3117 - val_loss: 1.7548 - val_acc: 0.3302
Epoch 3/10
28709/28709 [==============================] - 10s 345us/step - loss: 1.7097 - acc: 0.3491 - val_loss: 1.7328 - val_acc: 0.3845
Epoch 4/10
  768/28709 [..............................] - ETA: 9s - loss: 1.6740 - acc: 0.3984 

28709/28709 [==============================] - 10s 345us/step - loss: 1.6410 - acc: 0.3794 - val_loss: 1.5763 - val_acc: 0.3770
Epoch 5/10
28709/28709 [==============================] - 10s 344us/step - loss: 1.5694 - acc: 0.4035 - val_loss: 1.5746 - val_acc: 0.4093
Epoch 6/10
28709/28709 [==============================] - 10s 340us/step - loss: 1.5182 - acc: 0.4233 - val_loss: 1.4626 - val_acc: 0.4330
Epoch 7/10
11648/28709 [===========>..................] - ETA: 5s - loss: 1.4980 - acc: 0.4377

28709/28709 [==============================] - 10s 342us/step - loss: 1.4779 - acc: 0.4400 - val_loss: 1.4774 - val_acc: 0.4380
Epoch 8/10
28709/28709 [==============================] - 10s 347us/step - loss: 1.4255 - acc: 0.4600 - val_loss: 1.4522 - val_acc: 0.4525
Epoch 9/10
28709/28709 [==============================] - 10s 349us/step - loss: 1.3819 - acc: 0.4730 - val_loss: 1.4560 - val_acc: 0.4388
Epoch 10/10
13440/28709 [=============>................] - ETA: 5s - loss: 1.3400 - acc: 0.4885

28709/28709 [==============================] - 10s 344us/step - loss: 1.3509 - acc: 0.4842 - val_loss: 1.4344 - val_acc: 0.4508
--- 104.47879362106323 seconds ---


In [29]:
# Wino 1 Separable
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 13s 459us/step - loss: 2.0133 - acc: 0.2504 - val_loss: 1.7558 - val_acc: 0.2931
Epoch 2/10
28709/28709 [==============================] - 10s 350us/step - loss: 1.8040 - acc: 0.3142 - val_loss: 1.7831 - val_acc: 0.3232
Epoch 3/10
28709/28709 [==============================] - 10s 345us/step - loss: 1.6975 - acc: 0.3585 - val_loss: 1.7235 - val_acc: 0.3709
Epoch 4/10
  128/28709 [..............................] - ETA: 9s - loss: 1.8020 - acc: 0.3828

28709/28709 [==============================] - 10s 347us/step - loss: 1.6248 - acc: 0.3901 - val_loss: 1.6815 - val_acc: 0.3837
Epoch 5/10
28709/28709 [==============================] - 10s 346us/step - loss: 1.5606 - acc: 0.4072 - val_loss: 1.6298 - val_acc: 0.4107
Epoch 6/10
28709/28709 [==============================] - 10s 345us/step - loss: 1.5046 - acc: 0.4302 - val_loss: 1.5186 - val_acc: 0.4185
Epoch 7/10
10624/28709 [==========>...................] - ETA: 6s - loss: 1.4488 - acc: 0.4547

28709/28709 [==============================] - 10s 350us/step - loss: 1.4610 - acc: 0.4486 - val_loss: 1.5716 - val_acc: 0.4269
Epoch 8/10
28709/28709 [==============================] - 10s 350us/step - loss: 1.4148 - acc: 0.4682 - val_loss: 1.4896 - val_acc: 0.4366
Epoch 9/10
28709/28709 [==============================] - 10s 348us/step - loss: 1.3785 - acc: 0.4808 - val_loss: 1.4299 - val_acc: 0.4425
Epoch 10/10
12800/28709 [============>.................] - ETA: 5s - loss: 1.3371 - acc: 0.4895

28709/28709 [==============================] - 10s 349us/step - loss: 1.3455 - acc: 0.4874 - val_loss: 1.4069 - val_acc: 0.4595
--- 105.74680256843567 seconds ---


In [36]:
# Wino 1 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 21s 715us/step - loss: 1.9565 - acc: 0.2723 - val_loss: 1.7349 - val_acc: 0.3266
Epoch 2/5
27648/28709 [===========================>..] - ETA: 0s - loss: 1.7632 - acc: 0.3394

28709/28709 [==============================] - 15s 533us/step - loss: 1.7624 - acc: 0.3405 - val_loss: 2.6360 - val_acc: 0.2577
Epoch 3/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.6694 - acc: 0.3788 - val_loss: 1.7269 - val_acc: 0.3901
Epoch 4/5
15360/28709 [===============>..............] - ETA: 6s - loss: 1.6046 - acc: 0.4005

28709/28709 [==============================] - 15s 531us/step - loss: 1.6014 - acc: 0.3990 - val_loss: 1.6849 - val_acc: 0.3650
Epoch 5/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.5383 - acc: 0.4241 - val_loss: 1.5914 - val_acc: 0.4004
--- 84.23344898223877 seconds ---


In [41]:
# Wino 0 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 19s 666us/step - loss: 1.9578 - acc: 0.2694 - val_loss: 1.7939 - val_acc: 0.3073
Epoch 2/5
28544/28709 [============================>.] - ETA: 0s - loss: 1.7300 - acc: 0.3487

28709/28709 [==============================] - 15s 536us/step - loss: 1.7292 - acc: 0.3488 - val_loss: 1.6637 - val_acc: 0.3452
Epoch 3/5
28709/28709 [==============================] - 15s 535us/step - loss: 1.6358 - acc: 0.3831 - val_loss: 2.1404 - val_acc: 0.2792
Epoch 4/5
15104/28709 [==============>...............] - ETA: 7s - loss: 1.6028 - acc: 0.4033

28709/28709 [==============================] - 15s 533us/step - loss: 1.5859 - acc: 0.4056 - val_loss: 1.6214 - val_acc: 0.3773
Epoch 5/5
28709/28709 [==============================] - 15s 534us/step - loss: 1.5197 - acc: 0.4234 - val_loss: 1.5772 - val_acc: 0.3996
--- 82.82288789749146 seconds ---


In [46]:
# Wino 1 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 20s 681us/step - loss: 1.9462 - acc: 0.2755 - val_loss: 1.7434 - val_acc: 0.3266
Epoch 2/5
27648/28709 [===========================>..] - ETA: 0s - loss: 1.7649 - acc: 0.3430

28709/28709 [==============================] - 15s 538us/step - loss: 1.7648 - acc: 0.3434 - val_loss: 1.9530 - val_acc: 0.3380
Epoch 3/5
28709/28709 [==============================] - 15s 537us/step - loss: 1.6668 - acc: 0.3793 - val_loss: 1.5976 - val_acc: 0.3639
Epoch 4/5
15360/28709 [===============>..............] - ETA: 6s - loss: 1.6099 - acc: 0.4033

28709/28709 [==============================] - 15s 534us/step - loss: 1.5981 - acc: 0.4060 - val_loss: 1.6072 - val_acc: 0.3915
Epoch 5/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.5397 - acc: 0.4279 - val_loss: 1.6131 - val_acc: 0.4115
--- 83.71084761619568 seconds ---
